In [1]:
with open('input.txt', 'r') as f:
    lines = f.read().splitlines()

In [2]:
conditions = []
checks = []
for line in lines:
    condition, check = line.split()
    check = tuple([int(i) for i in check.split(',')])
    conditions.append(condition)
    checks.append(check)

# Part 1

In [3]:
def good_proposal(proposal, check):
    return check == tuple([len(g) for g in proposal.split('.') if len(g) > 0])

### Brute force with binary numbers

In [4]:
arrangement_sum = 0

for condition, check in zip(conditions, checks):

    # Number of unknown entries
    n_unknown = condition.count('?')

    # Number of possible arrangement proposals
    n_proposals = 2**n_unknown
    
    # We generate strings of length n_unknown consisitng of all possible 
    # combinations of zeros (., operational) and ones (#, broken)
    format_string = f"{{:0{n_unknown}b}}"

    # We can generate all proposals by looping over all possible numbers
    # between 0 and 2**n_unknown and formatting them as binary strings
    for i in range(n_proposals):

        binary_proposal = format_string.format(i)
        proposal = binary_proposal.replace('0', '.').replace('1', '#')

        condition_format = condition.replace('?', '{}')
        condition_proposal = condition_format.format(*[c for c in proposal])

        if good_proposal(condition_proposal, check):
            arrangement_sum += 1

print(arrangement_sum)

7599


### Slightly optmized with `itertools`

In [5]:
import itertools

arrangement_sum = 0

for condition, check in zip(conditions, checks):

    # Number of unknown entries
    n_unknown = condition.count('?')

    # Number of known broken springs
    n_broken = condition.count('#')

    # Total number of broken springs
    n_broken_total = sum(check)

    # Consider all combinations of n_broken_total - n_broken broken springs
    for configuration in itertools.combinations(
        range(n_unknown), n_broken_total - n_broken
        ):

        # Convert a tuple of indices into a string of dots and hashes
        proposal = ''.join(
            ['#' if i in configuration else '.' for i in range(n_unknown)]
            )

        condition_format = condition.replace('?', '{}')
        condition_proposal = condition_format.format(*[c for c in proposal])

        if good_proposal(condition_proposal, check):
            arrangement_sum += 1

print(arrangement_sum)

7599


### Much faster with a recursive function

In [6]:
from functools import cache

@cache
def count(condition, check):

    # We can use recursion to consider all choices for the unknown springs and
    # build up a count of the number of valid arrangements. The idea is to go
    # through each character of the condition string from left to right until 
    # we run our of characters in the conditions tring, or we run out of check
    # values. We will have a base case for each of these scenarios:
    
    if len(condition) == 0:
        # This is okay as long as all the numbers in check have been used up:
        if len(check) == 0:
            return 1
        # If there are still numbers in the check, then this is not a valid
        # arrangement:
        else:
            return 0
        
    if len(check) == 0:
        # This is okay as long as there are no more broken springs in the 
        # condition:
        if '#' not in condition:
            return 1
        # If there are still broken springs in the condition, then this is not
        # a valid arrangement:
        else:
            return 0

    n_arrangements = 0

    # Before the above base cases, we iterate through the condition string. If 
    # the character is a dot, we simply "chop off" the dot and call the count 
    # function on the remaining condition string. We can also capture the case 
    # where we replace a question mark with a dot in this way. 
    if condition[0] in '.?':
        n_arrangements += count(condition[1:], check)

    # If the character is a hash (or we choose the question mark to be a hash), 
    # we can potentially move to the end of the proceeding contiguous group of 
    # damaged springs (i.e., move along multiple characters in the condition). 
    # Then we will call the count function on the remaining condition string, 
    # with the first element of the check list removed.
    if condition[0] in '#?':
        
        if (
            # For this group of hashes to correspond to the first value in 
            # check, there can't a dot in the condition string before the 
            # expected end of the group of hashes:
            ('.' not in condition[:check[0]])

            and

            # We also need to make sure there are enough characters left in the
            # condition string to satisfy the expected length of the group of
            # hashes:
            (len(condition) >= check[0])

            and

            # For this group of hashes to be the correct length, we require
            # a dot (or a question mark that can be turned into a dot) at the 
            # end of the expected length. We could also be at the end of the 
            # line:
            ((check[0] == len(condition)) or (condition[check[0]] != '#'))
        ):
            n_arrangements += count(condition[check[0]+1:], check[1:])

    # The above will iterate until we either get through every character in 
    # the condition string, or we run out of numbers in the check. When that 
    # happens, the base cases at the top of the function will be triggered.
        
    return n_arrangements

In [7]:
arrangement_sum = 0

for condition, check in zip(conditions, checks):
    arrangement_sum += count(condition, check)

print(arrangement_sum)

7599


# Part 2

In [8]:
arrangement_sum = 0

for condition, check in zip(conditions, checks):
    condition += 4*('?' + condition)
    check *= 5
    arrangement_sum += count(condition, check)

print(arrangement_sum)

15454556629917
